In [1]:
# Manipulate the file system
import os
import shutil

# Copy dictionaries
import copy

# Display errors in realtime
import ipywidgets as widgets

# work with dates
import datetime
import arrow
import time

# For scrubbing PII
import hashlib

# Convert stored string representation of a list to a list
import ast

# Recurse through a directory tree and return file names with glob
import glob

# Decode and re-encode mangled Arabic file names
import codecs

# Connect to a SQLite database in a lazy manner.
import sqlalchemy
import dataset

# Enables opening and reading of Excel files
import openpyxl

# Translating variables, sheet names, and workbook names from Arabic
# This is NOT free to use.
from google.cloud import translate

# Set the environment variable for the Google Service Account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:\\Users\\clay\\Documents\\fxb-lcs-2b24f4f8a73a.json'

In [2]:
#If there's an existing db for this sheet, delete it
#so that we can copy from the template for a fresh start

try:
    os.remove("sams_data_phase18.sqlite")
    print("Removed template clone sams_data_phase18.sqlite")
except:
    pass

try:
    # Try to preserve a copy in case there is a problem and it has to be restored
    shutil.copy2("sams_data_phase18_template.sqlite","sams_data_phase18.sqlite")
    
    print("Created database from template: sams_data_phase18.sqlite")
except:
    pass

Created database from template: sams_data_phase18.sqlite


In [3]:
db = dataset.connect("sqlite:///sams_data_phase18.sqlite")

In [4]:
tab_arabic = db['arabic_values']
tab_tokens = db['arabic_tokens']

In [5]:
# Arabic Character Ranges Regex
# https://stackoverflow.com/questions/4446244/how-to-check-if-any-arabic-character-exists-in-the-string-javascript

translate_client = translate.Client()
target_lang = 'en'

In [6]:
record_counter = widgets.HTML(value="Records: 0",continuous_update=True)
character_counter = widgets.HTML(value="Characters: 0",continuous_update=True)
error_counter = widgets.HTML(value="Errors: 0",continuous_update=True)

### Translate the tokens

In [ ]:
character_count = 0
record_count = 0
error_count = 0

update_records = []

for row in tab_tokens.find():
    arabic_string = row['token']
    # arabic_string.replace("_"," ").replace("\n"," ").replace("  "," ").replace("\t"," ").strip()
    try:
        translation = translate_client.translate(arabic_string, target_language=target_lang)
        rec = {
            "id":row['id'],
            "google_translate_feb":translation['translatedText']
        }
        update_records.append(rec)
        character_count += len(row['token'])
        character_counter.value = "Characters: " + str(character_count)
        record_count += 1
        record_counter.value = "Records: " + str(record_count)

    except:
        error_count += 1
        error_counter.value = "Errors: " + str(error_count)
        time.sleep(10)

        
print("--------------\nDone")
print("Record Count",record_count)
print("Character Count",character_count)

In [8]:
record_counter

In [9]:
character_counter

In [10]:
error_counter

In [ ]:
# DO NOT FORGET TO UPDATE THE TABLE!
for record in update_records:
    tab_tokens.update(record,['id'])